In [1]:
import torch
from transformers import AutoTokenizer, AutoModel, AutoModelWithLMHead

In [2]:
tokenizer = AutoTokenizer.from_pretrained("sberbank-ai/rugpt3large_based_on_gpt2")

In [3]:
model = AutoModelWithLMHead.from_pretrained("sberbank-ai/rugpt3large_based_on_gpt2")

In [4]:
# encode context the generation is conditioned on
input_ids = tokenizer.encode('Я люблю гулять со своей собакой', return_tensors='pt')

In [31]:
def predict_token(s, n=5, l=1, top_k=0, top_p=1., temperature=1.):
    # encode context the generation is conditioned on
    input_ids = tokenizer.encode(s, return_tensors='pt')
    # activate beam search and early_stopping
    beam_outputs = model.generate(
        input_ids,  
        max_length=input_ids.shape[1]+l, 
#         num_beams=n, 
        early_stopping=True,
        do_sample=True,
        num_return_sequences=n,
        top_k=top_k,
        top_p=top_p,
        temperature=temperature
    ).to('cuda')

    # now we have 3 output sequences
    print(f"{s}:\n" + 100 * '-')
    for i, beam_output in enumerate(beam_outputs):
      print("{}: {}".format(i, tokenizer.decode(beam_output, skip_special_tokens=True)[len(s):]).strip())

In [6]:
predict_token('Я люблю')

Setting `pad_token_id` to 50256 (first `eos_token_id`) to generate sequence
C:\ProgramData\Anaconda3\lib\site-packages\torch\utils\checkpoint.py:25: UserWarning: None of the inputs have requires_grad=True. Gradients will be None
  warnings.warn("None of the inputs have requires_grad=True. Gradients will be None")


Я люблю:
----------------------------------------------------------------------------------------------------
0: ' тебя'
1: ','
2: ' вас'
3: ' свою'
4: ' его'


In [7]:
predict_token('Я хочу')

Setting `pad_token_id` to 50256 (first `eos_token_id`) to generate sequence


Я хочу:
----------------------------------------------------------------------------------------------------
0: ','
1: ' сказать'
2: ' быть'
3: ' знать'
4: ' жить'


In [8]:
predict_token('Он')

Setting `pad_token_id` to 50256 (first `eos_token_id`) to generate sequence


Он:
----------------------------------------------------------------------------------------------------
0: ' был'
1: ' не'
2: ' сказал'
3: ','
4: ' уже'


In [9]:
predict_token('Hello')

Setting `pad_token_id` to 50256 (first `eos_token_id`) to generate sequence


Hello:
----------------------------------------------------------------------------------------------------
0: ','
1: '!'
2: ' World'
3: '\n'
4: ' world'


In [12]:
predict_token('Я хочу', l=50, top_k=10, top_p=0.95) 

Setting `pad_token_id` to 50256 (first `eos_token_id`) to generate sequence


Я хочу:
----------------------------------------------------------------------------------------------------
0: ', чтобы ты была счастлива.\n\n—\xa0Я счастлива.\n\n—\xa0Я хочу, чтобы ты была счастлива.\n\n—\xa0Я счастлива.\n\n—\xa0Я хочу, чтобы ты была счастлива.\n\n—\xa0Я счастлива'
1: ', чтобы ты была счастлива.\n\n–\xa0Я счастлива.\n\n–\xa0Я хочу, чтобы ты была счастлива.\n\n–\xa0Я счастлива.\n\n–\xa0Я хочу, чтобы ты была счастлива.\n\n–\xa0Я счастлива'
2: ', чтобы ты была счастлива.\n\n—\xa0Я счастлива.\n\n—\xa0Я хочу, чтобы ты была счастлива.\n\n—\xa0Я счастлива.\n\n—\xa0Я хочу, чтобы ты была счастлива.\n\n—\xa0Я…'
3: ', чтобы ты была счастлива.\n\n—\xa0Я счастлива.\n\n—\xa0Я хочу, чтобы ты была счастлива.\n\n—\xa0Я счастлива.\n\n—\xa0Я хочу, чтобы ты была счастлива.\n\n—\xa0Я хочу'
4: ', чтобы ты была счастлива.\n\n–\xa0Я счастлива.\n\n–\xa0Я хочу, чтобы ты была счастлива.\n\n–\xa0Я счастлива.\n\n–\xa0Я хочу, чтобы ты была счастлива.\n\n–\xa0Я…'


In [29]:
predict_token('Вечерело. Проходя мимо старого засаленого кабака я вдруг', l=50, top_k=50, top_p=0.95, n=5)

Setting `pad_token_id` to 50256 (first `eos_token_id`) to generate sequence


Вечерело. Проходя мимо старого засаленого кабака я вдруг:
----------------------------------------------------------------------------------------------------
0:  поймал себя на мысли, что я не хочу выпить в этом захолустье, и это желание почему-то мне понравилось.

Проходя мимо какого-то грязного здания я остановился и подумал, что неплохо было бы помыться, как вдруг
1:  услышал скрип калитки, и увидел на ней силуэт. Это была женщина. Я быстро завернул за угол и увидел ее. Женщина была босая и в одной рубашке. Я увидел, как она сняла с головы свою соломенную шляпу, отбросила ее
2:  вспомнил:
- Ребята, а у Вас ничего не найдется выпить?
В ответ мне был гробовой взгляд.
Я тогда понял, что мне и здесь не рады.
В этот момент в кабаке появились люди в чёрных плащах.
3:  заметил, что в окне зажигается свет и кто-то выходит на улицу. Вскочив на свой велосипед я помчался на свет и через пару минут увидел того самого бродягу, который вышел за мной. Я его окликнул, но тот не обратил на меня
4: 

In [35]:
predict_token('Вечерело. Проходя мимо старого засаленого кабака я вдруг', l=50, top_k=50, top_p=0.90, n=5) 

Setting `pad_token_id` to 50256 (first `eos_token_id`) to generate sequence


Вечерело. Проходя мимо старого засаленого кабака я вдруг:
----------------------------------------------------------------------------------------------------
0:  подумал: «А может это тот самый? Надо бы наведаться. Только сегодня как-то не хочется, а вдруг это тот самый? Я ведь так ничего о нем не знаю. Хотя, чего это я все о нем? Да и к чему
1:  заметил двух проституток, они курили одну сигарету. Одна из них была явно с улицы и в платье, как у шлюхи. Я не мог не подойти, познакомиться, но она сделала мне знак молчать. И я стоял молча, разглядывая ее платье
2:  увидел, что на дверях стоит объявление, извещающее о закрытии, да еще и с указанием времени.
- Блин, ну неееет… - протянул я и полез в карман за мелочью.
- Деньги есть?
Я с
3:  остановился, и не просто остановился, а замер.

Там, за стойкой, кто-то сидел и пил. Я заглянул за стойку и замер от ужаса:

— Сюда нельзя, не смотри, — сказала Оля.
4:  остановился, будто налетел на стену, и долго пытался вспомнить, где же я видел его. 